<a href="https://www.kaggle.com/code/dennismathewjose/customer-segmentation-using-rfm-analysis?scriptVersionId=208408253" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-data/data.csv


## Data Preprocessing

In [2]:
filepath = '/kaggle/input/ecommerce-data/data.csv'
data = pd.read_csv(filepath, encoding = 'ISO-8859-1')
copy_data = data.copy()

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
data.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In [7]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
### Converting the Date Column to Date Type
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [10]:
#Handling the missing values and negative values appropriately
data.loc[data['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [11]:
data.loc[data['Quantity']  < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [12]:
cleaned_data = data.dropna(subset = ['CustomerID'])

In [13]:
cleaned_data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [14]:
cleaned_data.loc[data['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [15]:
cleaned_data = cleaned_data[cleaned_data['Quantity'] > 0]

In [16]:
cleaned_data.loc[cleaned_data['Quantity']  < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [17]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [18]:
cleaned_data.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397924.0,13.021823,1.0,2.0,6.0,12.0,80995.0,180.42021
InvoiceDate,397924,2011-07-10 23:43:36.912475648,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
UnitPrice,397924.0,3.116174,0.0,1.25,1.95,3.75,8142.75,22.096788
CustomerID,397924.0,15294.315171,12346.0,13969.0,15159.0,16795.0,18287.0,1713.169877


## Exploratory Data Visualization

In [19]:
unique_customers = cleaned_data['CustomerID'].nunique()
print(f"There are {unique_customers} unique customers.")

There are 4339 unique customers.


In [20]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

customer_order_counts = cleaned_data.groupby('CustomerID')['InvoiceNo'].nunique()
fig = px.histogram(customer_order_counts, nbins=30, title="Distribution of Orders per Customer")
fig.show()

In [21]:
top_customers = customer_order_counts.nlargest(5).reset_index()
print(top_customers)

fig = px.bar(top_customers, x='CustomerID', y='InvoiceNo', title="Top 5 Customers by Order Count")
fig.show()


   CustomerID  InvoiceNo
0     12748.0        210
1     14911.0        201
2     17841.0        124
3     13089.0         97
4     14606.0         93


In [22]:
top_products = cleaned_data['Description'].value_counts().head(10).reset_index()
top_products.columns = ['Description', 'Count']

fig = px.bar(top_products, x='Description', y='Count', title="Top 10 Most Frequently Purchased Products")
fig.show()

In [23]:
average_price = cleaned_data['UnitPrice'].mean()
print(f"The average product price is {average_price:.2f}.")


The average product price is 3.12.


In [24]:
cleaned_data['Revenue'] = cleaned_data['Quantity'] * cleaned_data['UnitPrice']
revenue_per_product = cleaned_data.groupby('Description')['Revenue'].sum().reset_index()
top_revenue_product = revenue_per_product.nlargest(1, 'Revenue')
print(top_revenue_product)

fig = px.bar(revenue_per_product.nlargest(10, 'Revenue'), x='Description', y='Revenue', title="Top Revenue-Generating Products")
fig.show()


                      Description   Revenue
2319  PAPER CRAFT , LITTLE BIRDIE  168469.6


In [25]:
cleaned_data['InvoiceDate'] = pd.to_datetime(cleaned_data['InvoiceDate'])
cleaned_data['DayOfWeek'] = cleaned_data['InvoiceDate'].dt.day_name()
day_order_counts = cleaned_data['DayOfWeek'].value_counts()

fig = px.bar(day_order_counts, x=day_order_counts.index, y=day_order_counts.values, title="Orders by Day of the Week")
fig.show()


In [26]:
cleaned_data['Month'] = cleaned_data['InvoiceDate'].dt.month
month_order_counts = cleaned_data['Month'].value_counts().sort_index()

fig = px.line(x=month_order_counts.index, y=month_order_counts.values, title="Orders by Month")
fig.update_xaxes(title="Month")
fig.update_yaxes(title="Order Count")
fig.show()


In [27]:
top_countries = cleaned_data['Country'].value_counts().head(5).reset_index()
top_countries.columns = ['Country', 'Count']

fig = px.bar(top_countries, x='Country', y='Count', title="Top 5 Countries by Number of Orders")
fig.show()


In [28]:
average_order_value = cleaned_data.groupby('Country')['Revenue'].mean().reset_index()

fig = px.bar(average_order_value, x='Country', y='Revenue', title="Average Order Value by Country")
fig.show()


## RFM Calculation

#### To calculate RFM (Recency, Frequency, and Monetary) metrics, follow these steps:

#### Steps to Calculate RFM:
#### 1. Prepare the Dataset
Ensure your dataset has the following columns:

##### CustomerID: Unique identifier for each customer.
##### InvoiceDate: Date of each transaction.
##### InvoiceNo: Unique identifier for each order.
##### Quantity: Number of items purchased.
##### UnitPrice: Price per unit of the product.
#### 2. Calculate RFM Metrics
##### Recency (R): Days since the last purchase.
##### Frequency (F): Total number of orders per customer.
##### Monetary (M): Total monetary value of purchases per customer.

In [29]:
import pandas as pd

# Add a total price column
cleaned_data['TotalPrice'] = cleaned_data['Quantity'] * cleaned_data['UnitPrice']

# Define a reference date for recency calculation (e.g., last transaction date in the dataset)
reference_date = cleaned_data['InvoiceDate'].max()

# Group by CustomerID to calculate RFM
rfm = cleaned_data.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (reference_date - x.max()).days),
    Frequency=('InvoiceNo', 'nunique'),
    Monetary=('TotalPrice', 'sum')
).reset_index()

# Display the RFM metrics
print(rfm.head())

   CustomerID  Recency  Frequency  Monetary
0     12346.0      325          1  77183.60
1     12347.0        1          7   4310.00
2     12348.0       74          4   1797.24
3     12349.0       18          1   1757.55
4     12350.0      309          1    334.40


### Explanation of the Code
#### Recency:
(reference_date - x.max()).days: Calculates the number of days since the last purchase for each customer.
#### Frequency:
'nunique' on InvoiceNo counts the number of unique orders per customer.
#### Monetary:
The sum of the TotalPrice column calculates the total revenue generated by each customer.

In [30]:
import plotly.express as px

# Recency Distribution
fig_recency = px.histogram(rfm, x='Recency', nbins=20, title='Distribution of Recency (R)')
fig_recency.update_xaxes(title='Days Since Last Purchase')
fig_recency.update_yaxes(title='Count of Customers')
fig_recency.show()


In [31]:

# Frequency Distribution
fig_frequency = px.histogram(rfm, x='Frequency', nbins=20, title='Distribution of Frequency (F)')
fig_frequency.update_xaxes(title='Number of Orders')
fig_frequency.update_yaxes(title='Count of Customers')
fig_frequency.show()


In [32]:
# Monetary Distribution
fig_monetary = px.histogram(rfm, x='Monetary', nbins=20, title='Distribution of Monetary Value (M)')
fig_monetary.update_xaxes(title='Total Monetary Value ($)')
fig_monetary.update_yaxes(title='Count of Customers')
fig_monetary.show()

## RFM Segmentation

#### Step 1: Assign RFM Scores
##### We will divide the Recency, Frequency, and Monetary metrics into quartiles (1–4):

- Recency: Lower scores (closer to 1) indicate recent purchases.
- Frequency: Higher scores (closer to 4) indicate frequent purchases.
- Monetary: Higher scores (closer to 4) indicate higher spending.

In [33]:
def assign_rfm_scores_with_cut(data, column, ascending=True):
    bins = pd.cut(data[column], bins=4, labels=[4, 3, 2, 1] if ascending else [1, 2, 3, 4])
    return bins

# Assign scores
rfm['R_Score'] = assign_rfm_scores_with_cut(rfm, 'Recency', ascending=False)
rfm['F_Score'] = assign_rfm_scores_with_cut(rfm, 'Frequency', ascending=True)
rfm['M_Score'] = assign_rfm_scores_with_cut(rfm, 'Monetary', ascending=True)


In [34]:
# Combine RFM scores into a single score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Display the first few rows
print(rfm[['CustomerID', 'R_Score', 'F_Score', 'M_Score', 'RFM_Score']].head())

   CustomerID R_Score F_Score M_Score RFM_Score
0     12346.0       4       4       3       443
1     12347.0       1       4       4       144
2     12348.0       1       4       4       144
3     12349.0       1       4       4       144
4     12350.0       4       4       4       444


#### Step 2: Analyze RFM Segments
##### After calculating the scores, you can group customers into meaningful segments:

- Example Segments
- Champions (RFM Score: 444): Recent, frequent, and high spenders.
- Loyal Customers: High frequency and monetary, but not necessarily recent.
- At Risk: Used to spend often but haven’t made recent purchases.
- Lost Customers (RFM Score: 111): Old purchases, infrequent, and low spenders.

In [35]:
# Define RFM segments based on RFM scores
def segment_rfm(score):
    if score == '444':
        return 'Champion'
    elif score.startswith('44'):
        return 'Loyal'
    elif score.endswith('44'):
        return 'Big Spender'
    elif score.startswith('11'):
        return 'Lost'
    else:
        return 'At Risk'

rfm['Segment'] = rfm['RFM_Score'].apply(segment_rfm)

# Display segment counts
print(rfm['Segment'].value_counts())

Segment
Big Spender    3974
Champion        346
At Risk          16
Lost              2
Loyal             1
Name: count, dtype: int64


### Visualizations

In [36]:
import plotly.express as px

fig = px.pie(rfm, names='Segment', title='RFM Segment Distribution', hole=0.4)
fig.show()

In [37]:
import plotly.express as px

# Count customers in each segment
segment_count = rfm['Segment'].value_counts().reset_index()
segment_count.columns = ['Segment', 'Customer Count']

# Plot the segments
fig = px.bar(segment_count, x='Segment', y='Customer Count', color='Segment', 
             title='Customer Segmentation Based on RFM Scores')
fig.show()

In [38]:
# Calculate mean R, F, M scores by segment
segment_metrics = rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].mean().reset_index()

# Plot the metrics
fig = px.bar(segment_metrics, x='Segment', y=['Recency', 'Frequency', 'Monetary'], 
             title='Average RFM Metrics by Segment', barmode='group')
fig.show()

In [39]:
rfm_scores = rfm.groupby(['R_Score', 'F_Score']).size().reset_index(name='Count')

fig = px.density_heatmap(rfm_scores, x='R_Score', y='F_Score', z='Count', 
                         title='Heatmap of RFM Scores', color_continuous_scale='Viridis')
fig.show()

/tmp/ipykernel_17/3609140261.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

